In [29]:
# Import required packages
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import urllib

In [28]:

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(url)

# Save the dataframe to a CSV file
spacex_df.to_csv('spacex_launch_dash.csv', index=False)


In [23]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"
filename = "spacex_dash_app.py"
urllib.request.urlretrieve(url, filename)


('spacex_dash_app.py', <http.client.HTTPMessage at 0x1c873df62d0>)

In [30]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [47]:
filtered_df = spacex_df[["class", "Launch Site"]]
data = filtered_df.groupby(["Launch Site"])["class"].value_counts()
data.head()

Launch Site   class
CCAFS LC-40   0        19
              1         7
CCAFS SLC-40  0         4
              1         3
KSC LC-39A    1        10
Name: count, dtype: int64

# Dashboard

In [62]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', options=[{'label': 'All Sites', 'value': 'ALL'},
                                {'label': 'CCAFS LC-40	', 'value': 'CCAFS LC-40'},
                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}],
                                    value = "ALL",
                                    placeholder = "Select Launch Site",
                                    searchable = True),
                                    html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min = 0, max = 10000, step = 1000,
                                                marks = {0: "0",
                                                         5000: "5000",
                                                         10000 : "10000"},
                                                         value = [min_payload, max_payload]
                                                         ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # Use all rows in the dataframe
        data = spacex_df['class'].value_counts().reset_index()
        data.columns = ['class', 'count']
        fig = px.pie(data, values='count', 
                     names='class', 
                     title='Total Success Launches By All Sites')
    else:
        # Filter the dataframe for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        data = filtered_df['class'].value_counts().reset_index()
        data.columns = ['class', 'count']
        fig = px.pie(data, values='count', 
                     names='class', 
                     title=f'Total Success Launches for site {entered_site}')
    return fig
# return the outcomes piechart for a selected site
# return the outcomes piechart for a selected site

# TASK 4:
@app.callback(Output(component:='success-payload-scatter-chart', component_property='figure'),
              Input(component_id = "payload-slider", component_property="value"),
              Input(component_id='site-dropdown', component_property='value'),
              )
def get_scatter_chart(entered_payload, entered_site):
    if entered_site == 'ALL':
        # Use all rows in the dataframe
        filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(entered_payload[0], entered_payload[1])]
        fig = px.scatter(filtered_df, x="Payload Mass (kg)", y="class", color="Booster Version Category",
                        title=f'Correlation between Payload and Success for all sites')
    else:
        # Filter the dataframe for the selected site
        filtered_df = spacex_df[(spacex_df['Launch Site'] == entered_site) & (spacex_df['Payload Mass (kg)'].between(entered_payload[0], entered_payload[1]))]
        fig = px.scatter(filtered_df, x="Payload Mass (kg)", y="class", color="Booster Version Category",
                        title=f'Correlation between Payload and Success for site {entered_site}')
    return fig
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output


# Run the app
if __name__ == '__main__':
    app.run_server()
